In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
 #With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.

train_subset = 10000
#we need beta for regularization later
beta = .01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  #now do regularization
  regularizer = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta*regularizer)
    
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 48.834435
Training accuracy: 10.0%
Validation accuracy: 11.9%
Loss at step 100: 11.694282
Training accuracy: 73.6%
Validation accuracy: 72.0%
Loss at step 200: 4.414845
Training accuracy: 79.3%
Validation accuracy: 76.4%
Loss at step 300: 1.950859
Training accuracy: 82.7%
Validation accuracy: 79.5%
Loss at step 400: 1.117104
Training accuracy: 84.2%
Validation accuracy: 81.2%
Loss at step 500: 0.831488
Training accuracy: 84.7%
Validation accuracy: 81.8%
Loss at step 600: 0.732412
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 700: 0.697702
Training accuracy: 84.9%
Validation accuracy: 82.1%
Loss at step 800: 0.685443
Training accuracy: 84.9%
Validation accuracy: 82.1%
Test accuracy: 88.9%


In [7]:
#stochastic version with regularization
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta*regularizer)
  
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 47.435043
Minibatch accuracy: 4.7%
Validation accuracy: 6.1%
Minibatch loss at step 500: 0.734320
Minibatch accuracy: 86.7%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.799276
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.566156
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.648464
Minibatch accuracy: 88.3%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.782524
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.784397
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Test accuracy: 88.6%


In [12]:
#SGD with relu and regularization
batch_size = 128
relu_num = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_num]))
  biases_1 = tf.Variable(tf.zeros([relu_num]))
  
  weights_2 = tf.Variable(tf.truncated_normal([relu_num,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  logits_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  logits_2 = tf.matmul(logits_1,weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(loss + beta * regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul( tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [13]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3453.218750
Minibatch accuracy: 11.7%
Validation accuracy: 22.1%
Minibatch loss at step 500: 21.191586
Minibatch accuracy: 89.1%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.944058
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.582736
Minibatch accuracy: 86.7%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.604634
Minibatch accuracy: 89.8%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.719760
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 0.761639
Minibatch accuracy: 83.6%
Validation accuracy: 83.4%
Minibatch loss at step 3500: 0.770482
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 4000: 0.665063
Minibatch accuracy: 86.7%
Validation accuracy: 83.8%
Minibatch loss at step 4500: 0.690252
Minibatch accuracy: 84.4%
Validation accuracy: 83.2%
Test

In [15]:
#restrict training data to show overfitting
num_steps = 3001

train_dataset_1 = train_dataset[:500,:]
train_labels_1 = train_labels[:500]


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_1.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_1[offset:(offset + batch_size), :]
    batch_labels = train_labels_1[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3429.099121
Minibatch accuracy: 12.5%
Validation accuracy: 31.5%
Minibatch loss at step 500: 21.013691
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 0.487693
Minibatch accuracy: 99.2%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 0.300036
Minibatch accuracy: 99.2%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 0.282159
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 2500: 0.281037
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 0.277306
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Test accuracy: 86.7%


In [17]:
#SGD with relu and regularization and dropout
batch_size = 128
relu_num = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_num]))
  biases_1 = tf.Variable(tf.zeros([relu_num]))
  weights_2 = tf.Variable(tf.truncated_normal([relu_num,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  logits_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  keep_prob = tf.placeholder("float")
  relu_layer_dropout = tf.nn.dropout(logits_1, keep_prob)  
  logits_2 = tf.matmul(relu_layer_dropout,weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(loss + beta * regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul( tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [19]:
#restrict training data to show overfitting
num_steps = 3001

train_dataset_1 = train_dataset[:500,:]
train_labels_1 = train_labels[:500]


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_1.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_1[offset:(offset + batch_size), :]
    batch_labels = train_labels_1[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3644.020996
Minibatch accuracy: 7.8%
Validation accuracy: 28.9%
Minibatch loss at step 500: 21.093777
Minibatch accuracy: 99.2%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 0.522143
Minibatch accuracy: 99.2%
Validation accuracy: 79.6%
Minibatch loss at step 1500: 0.332398
Minibatch accuracy: 100.0%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 0.314155
Minibatch accuracy: 99.2%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 0.307171
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 0.308483
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Test accuracy: 86.9%


In [20]:
#now use dropout on whole training set and see results

num_steps = 3001




with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3652.765869
Minibatch accuracy: 11.7%
Validation accuracy: 29.6%
Minibatch loss at step 500: 21.350695
Minibatch accuracy: 85.2%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 1.046307
Minibatch accuracy: 79.7%
Validation accuracy: 82.8%
Minibatch loss at step 1500: 0.644843
Minibatch accuracy: 86.7%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 0.702265
Minibatch accuracy: 86.7%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.795995
Minibatch accuracy: 83.6%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 0.856166
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Test accuracy: 89.9%


In [38]:
#SGD with relu and regularization and dropout, plus adding more layers and learning rate decay
batch_size = 128
relu_num = 1024
layer_3_num = 512
layer_4_num = 256
beta = .001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_num]))
  biases_1 = tf.Variable(tf.zeros([relu_num]))
  weights_2 = tf.Variable(tf.truncated_normal([relu_num,layer_3_num]))
  biases_2 = tf.Variable(tf.zeros([layer_3_num]))
  weights_3 = tf.Variable(tf.truncated_normal([layer_3_num,num_labels]))
  biases_3 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  logits_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  keep_prob = tf.placeholder("float")
  relu_layer_dropout = tf.nn.dropout(logits_1, keep_prob)  
  
  logits_2 = tf.matmul(relu_layer_dropout,weights_2) + biases_2
  relu_layer2_dropout
  logits_3 = tf.matmul(logits_2, weights_3) + biases_3
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits_3))
  regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3)
  loss = tf.reduce_mean(loss + beta * regularizers)

  # Optimizer with learning rate decay

  global_step = tf.Variable(0)  # count the number of steps taken.
  start_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 10000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(start_learning_rate).minimize(loss, global_step=global_step)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_3)

  #predictions for valid
  logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  relu_layer= tf.nn.relu(logits_1)
  logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
  logits_3 = tf.matmul(logits_2,weights_3)+biases_3
  valid_prediction = tf.nn.softmax(logits_3)
      
  # Predictions for test
  logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  relu_layer= tf.nn.relu(logits_1)
  logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
  logits_3 = tf.matmul(logits_2,weights_3)+biases_3
  test_prediction =  tf.nn.softmax(logits_3)

In [39]:
num_steps = 3001




with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 12700.284180
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 500: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%


KeyboardInterrupt: 